### Welcome to the cleaning experimentation notebook

Welcome to the cleaning experimentation notebook, this book is no long being worked on, but is a document to our approach.


In [1]:
# importing all the libraries i will need
import numpy as np
import pandas as pd
import sklearn as sl
import folium as fl
import csv
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# loading in the dataset for the sample
# updated to load the filtered dataset
try:
    dataset = pd.read_csv('datasets/crimedata2016.csv')
except:
    dataset = pd.read_csv('datasets/Chicago_Crimes_2012_to_2017.csv')
dataset.head()

,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,FBI Code,X Coordinate,Y Coordinate,Year,Location,Latitude,Longitude
0,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,08B,1154907.0,1893681.0,2016,"[41.864073157, -87.706818608]",41.864073,-87.706819
1,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,08B,1183066.0,1864330.0,2016,"[41.782921527, -87.60436317]",41.782922,-87.604363
2,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,24,1140789.0,1904819.0,2016,"[41.894908283, -87.758371958]",41.894908,-87.758372
3,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False,1532,08B,1143223.0,1901475.0,2016,"[41.885686845, -87.749515983]",41.885687,-87.749516
4,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,06,1139890.0,1901675.0,2016,"[41.886297242, -87.761750709]",41.886297,-87.761751


In [3]:
# dropping all information not needed by our program.
try:
    dataset = dataset.drop(['Unnamed: 0', 'ID', 'Case Number', 'Block', 'IUCR', 'Updated On', 'Latitude', 
            'Longitude', 'District', 'Ward', 'Community Area'], axis=1)
except:
    pass

# limiting our data to just a year.
dataset_train = dataset[dataset['Year'] == 2016]
dataset_test = dataset[dataset['Year'] == 2015]
dataset

,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,FBI Code,X Coordinate,Y Coordinate,Year,Location,Latitude,Longitude
0,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,08B,1154907.0,1893681.0,2016,"[41.864073157, -87.706818608]",41.864073,-87.706819
1,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,08B,1183066.0,1864330.0,2016,"[41.782921527, -87.60436317]",41.782922,-87.604363
2,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,24,1140789.0,1904819.0,2016,"[41.894908283, -87.758371958]",41.894908,-87.758372
3,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False,1532,08B,1143223.0,1901475.0,2016,"[41.885686845, -87.749515983]",41.885687,-87.749516
4,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,06,1139890.0,1901675.0,2016,"[41.886297242, -87.761750709]",41.886297,-87.761751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250727,05/03/2016 11:33:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1034,08B,1159105.0,1888300.0,2016,"[41.849222028, -87.69155551]",41.849222,-87.691556
250728,05/03/2016 11:30:00 PM,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,True,True,731,14,1175230.0,1856183.0,2016,"[41.760743949, -87.63333531]",41.760744,-87.633335
250729,05/03/2016 12:15:00 AM,BATTERY,AGGRAVATED: HANDGUN,SIDEWALK,False,False,825,04B,1161027.0,1862810.0,2016,"[41.779234743, -87.685207125]",41.779235,-87.685207
250730,05/03/2016 09:07:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,423,08B,1197261.0,1850727.0,2016,"[41.745251975, -87.552773464]",41.745252,-87.552773


In [4]:
# script to save all the data, for import and analysis later
# NOT NEEDED TO BE RUN ANYMORE

f = open('datasets/crimedata2016.csv', 'w')
writer = csv.writer(f)
writer.writerow(['Date', 'Primary Type', 'Description', 'Location Description', 'Arrest',
                'Domestic', 'Beat', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year',
                'Location', 'Latitude', 'Longitude'])
for i in range(len(dataset_train)):
    writer.writerow(dataset_train.iloc[i])
f.close()
# f = open('datasets/crimedata2017.csv', 'w')
# writer = csv.writer(f)
# writer.writerow(['Date', 'Primary Type', 'Description', 'Location Description', 'Arrest',
#                 'Domestic', 'Beat', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year',
#                 'Location'])
# for i in range(len(dataset_test)):
#     writer.writerow(dataset_test.iloc[i])
# f.close()

In [5]:
basicCrime = list(set(dataset_train['Primary Type']))

In [6]:
# convert location into lat, long lists.
locset = []
loclat = []
loclong = []
dataset_train = dataset_train.dropna()
for i in dataset_train['Location']:
    locset.append([float(i[0]), float(i[1])])
    loclat.append(float(i[0]))
    loclong.append(float(i[1]))

dataset_train['Location'] = locset
dataset_train['Latitude'] = loclat
dataset_train['Longitude'] = loclong
dataset_train.head()

ValueError: could not convert string to float: '['

In [34]:
df = pd.read_csv('datasets/Los AngelesRAW.csv')
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,201220752,09/16/2020 12:00:00 AM,09/16/2020 12:00:00 AM,1230,12,77th Street,1259,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,NaN,NaN,NaN,700 E 73RD ST,NaN,33.9739,-118.2630
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [35]:
try:
    df = df.drop(['DR_NO', 'Date Rptd', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Vict Age',
       'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Crm Cd 2', 'Crm Cd 3',
       'Crm Cd 4'], axis='columns')
except:
    pass
df.head()

,DATE OCC,TIME OCC,Crm Cd Desc,Mocodes,Status,Status Desc,Crm Cd 1,LOCATION,Cross Street,LAT,LON
0,01/08/2020 12:00:00 AM,2230,BATTERY - SIMPLE ASSAULT,0444 0913,AO,Adult Other,624.0,1100 W 39TH PL,NaN,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,330,BATTERY - SIMPLE ASSAULT,0416 1822 1414,IC,Invest Cont,624.0,700 S HILL ST,NaN,34.0459,-118.2545
2,09/16/2020 12:00:00 AM,1230,VANDALISM - MISDEAMEANOR ($399 OR UNDER),2004 1820 0913 0329 1202,IC,Invest Cont,745.0,700 E 73RD ST,NaN,33.9739,-118.2630
3,01/01/2020 12:00:00 AM,1730,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,IC,Invest Cont,745.0,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,415,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,IC,Invest Cont,740.0,14400 TITUS ST,NaN,34.2198,-118.4468


In [38]:
years = pd.Series([i[6:10] for i in df['DATE OCC']])
fldf = df.loc[(years == '2020')].reset_index()
cleandf = fldf.drop(['index'], axis=1)
cleandf.head()
print(len(df) - len(cleandf))

79812
